# FSI TimeMarching test

In [1]:
include("../src/FSI.jl")
using FSI

In [2]:
using Plots
pyplot()
clibrary(:colorbrewer)
default(grid = false)

## <span style="color:blue"> Set up rigid body with Dyn3d

#### Include any joint-body setup script from Dyn3d

In [3]:
include(Pkg.dir("Dyn3d")*"/src/config_files/2dFall.jl")

Config info set up.


#### Build joint-body chain

In [4]:
bs, js, bsys = BuildChain(config_bodys, config_joints, config_system)
bd = BodyDyn(bs, js, bsys)

#### Initialize rigid body system state to complete body-joint information

In [5]:
bd, soln₀ = InitSystem!(bd)

This is a 1 body-joint system, system is fixed in space

(, Dyn3d.ConstructSystem.Soln{Float64}(0.0, 0.001, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float64[], Float64[]))

This is a 1 body-joint system, system is fixed in space

#### Initialize u and λ container

In [6]:
qJ = soln₀.qJ
v = soln₀.v
λ_dim = bd.sys.ncdof_HERK
λ = zeros(Float64,λ_dim);

#### Get body verts for plotting

In [7]:
@get bd (bs, js, sys)
bs, js, sys = UpdatePosition!(bs, js, sys, soln₀.qJ)
vs₀ = VertsHistory(sys.nbody, bs);

#### Plot body only

In [8]:
# p = plot()
# for i = 1:sys.nbody
#     plot!(p,vs₀[i,2:3,1], vs₀[i,2:3,2],linewidth=2)
# end
# plot!(xlims=(0,3), ylims=(0,2))

## <span style="color:blue"> Set up fluid with Whirl </span>

#### Set the flow parameters

In [9]:
Re = 200 # Reynolds number
U = 1.0 # Free stream velocity
U∞ = (0.0, U)

(0.0, 1.0)

#### Set the domain grid

In [10]:
nx = 152; ny = 102;
Ly = 2.0;
Δx = Ly/(ny-2);
Δt = min(0.5*Δx,0.5*Δx^2*Re)
w₀ = Nodes(Dual,(nx,ny))
xg, yg = coordinates(w₀,dx=Δx)

(-0.01:0.02:3.0100000000000002, -0.01:0.02:2.0100000000000002)

#### Set up fluid initial conditions

In [11]:
w₀ .= 0.0;
w = deepcopy(w₀);

## <span style="color:blue"> Create fluid-body interface </span>

#### Genrate body grid points and get linear velocity on them

In [12]:
# bgs short for body grid system
bgs = GenerateBodyGrid(bd; np=DetermineNP(nbody, Δx))
bgs = CutOut2d(bd,bgs);

#### Get body points initial coordinates for fluids

In [13]:
bgs = AcquireBodyGridKinematics(bd,bgs)
coord_init = hcat(bgs[1].q_i...)'[:,[1,2]]
for i = 2:length(bgs)
    coord_init = [coord_init[1:end-1,:]; hcat(bgs[i].q_i...)'[:,[1,2]]]
end

#### Set up body grid points and force container in fluid

In [14]:
X̃ = VectorData(coord_init)
f = VectorData(X̃);

#### Create fluidstruct structure

In [15]:
tf = Δt
T = Δt:Δt:tf
fsys = FluidStruct((nx,ny),Re,Δx,Δt,U∞ = U∞, X̃ = X̃, rk=RK31)
N = length(X̃)÷2

51

## <span style="color:blue"> Create ifherk object </span>

In [ ]:
@time ifherk_coupled = FSI.IFHERK_coupled(fsys.Δt,bd,bgs,(w₀,qJ,v,f,λ),
                ((t,u) -> Systems.plan_intfact(t,u,fsys), B₁ᵀ, B₂),
                (M⁻¹, G₁ᵀ, G₂, UpP, UpV),
                ((bd,bgs,f) -> T₁ᵀ(bd,bgs,f,fsys.Δx), T₂, getX̃),
                ((w,t) -> r₁(w,t,fsys), F, gti),
                fsys, tol=1e-3,rk=Whirl.TimeMarching.RK31)

In [ ]:
fx = Float64[]
fy = Float64[]
thist = []
whist = []
bdhist = BodyDyn[];

## <span style="color:blue"> TimeMarching! </span>

In [ ]:
@time for t in T
    
    t, (w, f), (qJ, v, λ), bd = ifherk_coupled(t, w, qJ, v, bd,fsys)
    
    push!(thist,t)
    push!(fx,sum(f.u)*Δx^2)
    push!(fy,sum(f.v)*Δx^2)
    push!(whist,deepcopy(w))
    push!(bdhist,deepcopy(bd))
    println("next time unit is ",t)
end

## <span style="color:blue"> Plot </span>

In [ ]:
bs, js, sys = UpdatePosition!(bd.bs, bd.js, bd.sys, qJ)
vshist = VertsHistory(bd.sys.nbody, bs);

In [ ]:
p = plot(xg,yg,w)
for i = 1:bd.sys.nbody
    plot!(p,vshist[i,2:3,1], vshist[i,2:3,2],linewidth=2,linecolor="grey")
end
p

In [ ]:
px = plot(thist,2*fx,xlabel="Convective time",ylabel="\$C_D\$",legend=false)
py = plot(thist,2*fy,xlabel="Convective time",ylabel="\$C_L\$",legend=false)
plot(px,py)

In [ ]:
w[50:70,50:52]

In [ ]:
2*fx

In [ ]:
2*fy